In [51]:
import gensim, re
import numpy as np
import pandas as pd
import pickle
from os import listdir

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import sys
import os

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Embedding

from underthesea import word_tokenize

sep = os.sep
data_folder = "data"

In [52]:
df = pd.read_csv(data_folder + sep + 'Data.csv')
display(df)
display(df.describe())

,code,content,tag,reply
0,#cfs13921,"Chào mọi người, mình sinh năm 99, mới ra trườn...",NaN,NaN
1,#cfs13920,"""Đông đã về và xuân sắp tới. Nay mình mạo muội...",NaN,NaN
2,#Cfs13919,Em lại là thằng nhóc có crush theo người khác ...,NaN,NaN
3,#Cfs13918_1,Em chào mụi người. Hiện tại em đang là sinh vi...,#TìmNgườiYêu,NaN
4,#Cfs13918_2,Xin chàooo\nMọi người có thấy trời SG hôm nay ...,#TìmNgườiYêu,NaN
...,...,...,...,...
141,#cfs13891,Nợ người một lời giải thích\r\nBạn dạo này thế...,#tâmsự,NaN
142,#cfs13890,"Em chào anh chị, hiện em đang học khoa Hoá trư...",#tâmsự,NaN
143,#cfs13889,"Dạ hello mọi ngừi, em là k21 khoa mt, chuyện l...",#tâmsự,NaN
144,#cfs13888,Thấm thoát thôi mà đã sắp tròn một năm ngày cu...,#tâmsự,NaN


,code,content,tag,reply
count,146,146,125,16
unique,126,133,35,16
top,#cfs13905,Nếu bạn không có trách nhiệm với điểm số của m...,#tâmsự,"Em yêu nhiều anh lắm, anh là anh nào v"
freq,3,2,19,1


In [53]:
df['tag'] = df['tag'].fillna("#LCD")
display(df)
# df['tag'].value_counts()

,code,content,tag,reply
0,#cfs13921,"Chào mọi người, mình sinh năm 99, mới ra trườn...",#LCD,NaN
1,#cfs13920,"""Đông đã về và xuân sắp tới. Nay mình mạo muội...",#LCD,NaN
2,#Cfs13919,Em lại là thằng nhóc có crush theo người khác ...,#LCD,NaN
3,#Cfs13918_1,Em chào mụi người. Hiện tại em đang là sinh vi...,#TìmNgườiYêu,NaN
4,#Cfs13918_2,Xin chàooo\nMọi người có thấy trời SG hôm nay ...,#TìmNgườiYêu,NaN
...,...,...,...,...
141,#cfs13891,Nợ người một lời giải thích\r\nBạn dạo này thế...,#tâmsự,NaN
142,#cfs13890,"Em chào anh chị, hiện em đang học khoa Hoá trư...",#tâmsự,NaN
143,#cfs13889,"Dạ hello mọi ngừi, em là k21 khoa mt, chuyện l...",#tâmsự,NaN
144,#cfs13888,Thấm thoát thôi mà đã sắp tròn một năm ngày cu...,#tâmsự,NaN


In [54]:
def txtTokenizer(texts):
    tokenizer = Tokenizer()
    # fit the tokenizer on our text
    tokenizer.fit_on_texts(texts)

    # get all words that the tokenizer knows
    word_index = tokenizer.word_index
    return tokenizer, word_index

In [55]:
# def preProcess(sentences):
#     print(sentences)
#     text = [word_tokenize(sentence, format="text") for sentence in sentences]
#     print("-" * 40)
#     print(text)
#     text = [re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", '', sentence) for sentence in text if sentence!='']
#     text = [re.sub(r'([^\s\w]|\'")+', '', sentence) for sentence in text if sentence!='']
#     print("-" * 40)
#     print(text)
#     text = [sentence.lower().strip().split() for sentence in text]
#     return text

In [56]:
def preProcess(sentences):
#     print(sentences)
#     print("-" * 20)
    text = [word_tokenize(sentence, format="text") for sentence in sentences]
#     print(text)
#     text = [re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", '', sentence) for sentence in sentences if sentence!='']
    text = [re.sub(r'([^\s\w]|"\')+', '', sentence) for sentence in text if sentence!='']
#     print("-" * 30)
#     print(text)
    text = [sentence.lower().strip().split() for sentence in text]
#     print("-" * 40)
#     print(text)
    return text

In [57]:
print(df['content'][1])
some_sentences = df['content'][1].split('.')
some_sentences = preProcess(some_sentences)
# print(some_sentences)

"Đông đã về và xuân sắp tới. Nay mình mạo muội lên đây để tìm cho mình 1 cô bạn gái xinh xắn dễ thương để cùng nhau đón tết này.
GT về bản thân 1 tý thì mình là nam đang là sv BK năm 3 ở 1 khoa nào đó =)) Về ngoại hình thi tuy không có gym nhưng nhìn chung cũng tạm ổn =)) hehe. Sở thích thì thể thao, đi chơi ăn uống các thứ. Mình không dám yêu cầu quá nhiều về bạn nữ, chỉ mong đơn giản bạn nữ là người hiểu chuyện biết thông cảm chia sẻ với mình, vậy là được rồi. Mong mình sẽ tìm được nhau. Bạn nào cũng có ý định như mình thì hãy cho mình xin ít info nhe. mình sẽ liên lạc với bạn sớm nhất. 
Peaceee!!!
https://docs.google.com/.../1FAIpQLSflIqrzuoBSR.../viewform"


In [58]:
freq = dict({("#tìmngườiyêu", 4), ("#lcd", 9), ("#gópý", 11), ("#bócphốt", 12), ("#hỏiđáp", 13), ("#tìmbạn", 23), ("#tâmsự", 1), ("#chiasẻ", 1)})
print(freq)

{'#chiasẻ': 1, '#lcd': 9, '#tâmsự': 1, '#tìmngườiyêu': 4, '#bócphốt': 12, '#tìmbạn': 23, '#gópý': 11, '#hỏiđáp': 13}


In [59]:
def loadData():
    texts = []
    labels = []
    for sample in df['content']:
        sentences = sample.split('.')
        sentences = preProcess(sentences)
        tag = df.loc[df.content == sample, 'tag'].values[0]
        label = [preProcessTag(tag) for _ in sentences]
        # [tag tag tag tag tag]
        for i in range(freq[preProcessTag(tag)]):
            texts = texts + sentences
            labels = labels + label
    return texts, labels

######### TODO #########
# cách 1:
# bỏ dòng sample.split('.') thay sentences bằng sample
# sửa hàm preProcess vì trước đó nó xử lý list của những câu trong cùng 1 cfs 
# (mà giờ là xử lý string đại diện cho 1 cfs)
# MỤC ĐÍCH CUỐI: xử lý cfs theo đoạn thay vì theo dòng

# cách 2:
# break down cfs thành các câu rồi predict từng câu

# re.split('; |, |\*|\n',sample)

# ví dụ
# >> a='Beautiful, is; better*than\nugly'
# >> import re
# >> re.split('; |, |\*|\n',a)
# ['Beautiful', 'is', 'better', 'than', 'ugly']

In [60]:
def preProcessTag(tag):
    temp = tag.lower().replace(" ", "")
    if "ngườiyêu" in temp:
        return "#tìmngườiyêu"
    elif "tâmsự" in temp:
        return "#tâmsự"
    elif "gópý" in temp:
        return "#gópý"
    elif "bócphốt" in temp:
        return "#bócphốt"
    elif "hỏiđáp" in temp:
        return "#hỏiđáp"
    elif "bạn" in temp or "info" in temp or "ngườiđichơi" in temp:
        return "#tìmbạn"
    elif "chiasẻ" in temp:
        return "#chiasẻ"
    elif "lcd" in temp:
        return "#lcd"
    else:
        return "error"

In [ ]:
# dictionary = dict()
# for tag in Y_train.columns:
#     result = preProcessTag(tag)
#     if tag == "error":
#         print(tag)
#     dictionary[result] = dictionary.get(result, 0) + 1
# print(dictionary)

In [28]:
print("Data file not found, build it!")

texts, labels = loadData()
tokenizer, word_index = txtTokenizer(texts)

# put the tokens in a matrix
X = tokenizer.texts_to_sequences(texts)
print(X)
X = pad_sequences(X, maxlen=500)

# prepare the labels
Y = pd.get_dummies(labels)
file = open(data_folder + sep + "data_6.pkl", 'wb')
pickle.dump([X,Y, texts],file)
file.close()

Data file not found, build it!
[[109, 40, 9, 1, 368, 45, 980, 80, 63, 52, 21, 62, 75, 5, 752, 533, 1092, 1093, 31, 674, 89, 102, 72, 268], [1573, 67, 7, 1, 20, 1094, 1179, 659, 1574, 489, 227, 1575], [1, 65, 804, 30, 118, 6, 1540, 25, 611, 8, 117, 981, 351, 73, 1095, 31, 1, 99, 4, 119, 871, 66, 1, 2, 11], [42, 4, 248, 383, 1, 34, 702], [109, 40, 9, 1, 368, 45, 980, 80, 63, 52, 21, 62, 75, 5, 752, 533, 1092, 1093, 31, 674, 89, 102, 72, 268], [1573, 67, 7, 1, 20, 1094, 1179, 659, 1574, 489, 227, 1575], [1, 65, 804, 30, 118, 6, 1540, 25, 611, 8, 117, 981, 351, 73, 1095, 31, 1, 99, 4, 119, 871, 66, 1, 2, 11], [42, 4, 248, 383, 1, 34, 702], [109, 40, 9, 1, 368, 45, 980, 80, 63, 52, 21, 62, 75, 5, 752, 533, 1092, 1093, 31, 674, 89, 102, 72, 268], [1573, 67, 7, 1, 20, 1094, 1179, 659, 1574, 489, 227, 1575], [1, 65, 804, 30, 118, 6, 1540, 25, 611, 8, 117, 981, 351, 73, 1095, 31, 1, 99, 4, 119, 871, 66, 1, 2, 11], [42, 4, 248, 383, 1, 34, 702], [109, 40, 9, 1, 368, 45, 980, 80, 63, 52, 21, 62, 

In [29]:
print(tokenizer.sequences_to_texts(X))

['chào mọi người mình sinh năm 99 mới ra trường mà vẫn chưa có mảnh tình vắt vai nên mạo_muội lên đây tìm người_yêu', 'sơ qua thì mình học it việc_làm ổn_định thu_nhập 4 số tiểu_học', 'mình khá tự_ti về việc không đạt 1 m7 và hơi béo 6 x kg nên mình mong bạn nữ thấp hơn mình là được', 'các bạn thả tim mình sẽ inbox', 'chào mọi người mình sinh năm 99 mới ra trường mà vẫn chưa có mảnh tình vắt vai nên mạo_muội lên đây tìm người_yêu', 'sơ qua thì mình học it việc_làm ổn_định thu_nhập 4 số tiểu_học', 'mình khá tự_ti về việc không đạt 1 m7 và hơi béo 6 x kg nên mình mong bạn nữ thấp hơn mình là được', 'các bạn thả tim mình sẽ inbox', 'chào mọi người mình sinh năm 99 mới ra trường mà vẫn chưa có mảnh tình vắt vai nên mạo_muội lên đây tìm người_yêu', 'sơ qua thì mình học it việc_làm ổn_định thu_nhập 4 số tiểu_học', 'mình khá tự_ti về việc không đạt 1 m7 và hơi béo 6 x kg nên mình mong bạn nữ thấp hơn mình là được', 'các bạn thả tim mình sẽ inbox', 'chào mọi người mình sinh năm 99 mới ra trườn

In [ ]:
# print("Data file found, load it!")
# #
# file = open(data_folder + sep + "data_5.pkl", 'rb')
# X,Y,texts = pickle.load(file)
# file.close()

In [30]:
# split in train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=True)
# print(X)
print(type(X))
# print(tokenizer.sequences_to_texts(X))
# print(Y)

<class 'numpy.ndarray'>


In [31]:
# train Word2Vec model on our data
word_model = gensim.models.Word2Vec(texts, vector_size=300, min_count=1, epochs=10)
word_model.save(data_folder + sep + "word_model_6.save")

In [13]:
# load Word2Vec model on our data
word_model = gensim.models.Word2Vec.load(data_folder + sep + "word_model_6.save")

In [32]:
# check the most similar word to 'cơm'
print(word_model.wv.most_similar('em'))

[('review', 0.6324798464775085), ('ạ', 0.6139950156211853), ('anh', 0.6127460598945618), ('sợ', 0.5857173800468445), ('gởi', 0.5749900341033936), ('ngành', 0.5665122866630554), ('ngần_ngại', 0.5535144209861755), ('vẽ', 0.5496601462364197), ('con_gái', 0.5443174242973328), ('bảo_lưu', 0.5397445559501648)]


In [33]:
# Train model
embedding_matrix = np.zeros((len(word_model.wv) + 1, 300))
for i, vec in enumerate(word_model.wv.vectors):
    embedding_matrix[i] = vec

model = Sequential()
model.add(Embedding(len(word_model.wv)+1,300,input_length=X.shape[1],weights=[embedding_matrix],trainable=False))
model.add(LSTM(300,return_sequences=False))
model.add(Dense(Y.shape[1],activation="softmax"))
model.summary()
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

batch = 64
epochs = 100
model.fit(X_train,Y_train,batch,epochs)
model.save(data_folder + sep + "predict_model_6.save")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 272, 300)          938100    
                                                                 
 lstm_1 (LSTM)               (None, 300)               721200    
                                                                 
 dense_1 (Dense)             (None, 8)                 2408      
                                                                 
Total params: 1,661,708
Trainable params: 723,608
Non-trainable params: 938,100
_________________________________________________________________
81/81 [==============================] - 61s 728ms/step - loss: 1.2982 - acc: 0.5225


INFO:tensorflow:Assets written to: data\predict_model_6.save\assets


INFO:tensorflow:Assets written to: data\predict_model_6.save\assets


In [34]:
model.evaluate(X_test,Y_test)

18/18 [==============================] - 4s 205ms/step - loss: 0.8802 - acc: 0.6760


[0.8802374005317688, 0.6760070323944092]

In [ ]:
# Load model
model = load_model(data_folder + sep + "predict_model_6.save")
model.summary()

In [ ]:
model.fit(X_train,Y_train,batch,1)

In [ ]:
model.evaluate(X_train, Y_train)

In [35]:
import numpy as np

In [95]:
# Test model
file = open(data_folder + sep + "My_data.txt", "r", encoding="utf8")
# input_string = [file.read()]
input_string = file.read().split('.')
# input_string = [input_string[0] + "\n" + input_string[0]]
# print(input_string)
# print("-"*40)
# input_string = ["chào", "chào chào", "Em", "Em em", "chào em", "em chào", "chào chào em em", "chào chào em ạ"]
# print(txtTokenizer(preProcess(input_string))[0])
X_dev = tokenizer.texts_to_sequences(preProcess(input_string))
print(tokenizer.sequences_to_texts(X_dev))
X_dev = pad_sequences(X_dev, maxlen=len(X_test[0]))
# X_dev = pad_sequences(X_dev, maxlen=500)
# print(X_dev)
# print(X_test)
# print(len(X_dev[0]))
# print(len(X_test[2]))
print("*" * 40)
print(Y_train.columns)
print("*" * 40)
print(Y_train['#lcd'].value_counts())
print("*" * 40)
print(X_train)
print("Predicting...")
# prediction = model.predict(X_train, verbose=0)
# print(prediction)
# for i in range(len(X_train)):
#     if i < 10:
#         print(Y_train.columns[np.argmax(prediction[i])])
#         print(tokenizer.sequences_to_texts(X_train)[i])

['dạ mọi người ơi em k21 ạ', 'em muốn hỏi về chuyện học_bổng ban_đầu mới vào trường em nghe nói là điểm tb từ 7 là có tư_cách xét học_bổng', 'nhưng giờ mới biết thêm là số học_bổng là mỗi khoa xét riêng và lấy từ trên xuống', 'với sức học của em hiện_tại ráng lắm chắc cũng chỉ với đc tới 7 chấm thôi mà cái khoa em máy_tính thì toàn các vị thần không', 'các anh_chị khoá trước có_thể cho em biết là tầm bao_nhiêu điểm là mình có_thể chắc ăn có được học_bổng không ạ rồi cách ôn_thi cuối kì hiệu_quả vì điểm giữa kỳ em k được tốt lắm nên em sẽ cố_gắng hết_sức cho cuối kỳ', 'những comment góp_ý em xin chân_thành ghi_nhận ạ', 'còn về bài_ca học bách_khoa phải rớt môn học_lại thì thôi ạ vì em không muốn làm phí thêm tiền của ba_mẹ ạ', 'em cảm_ơn']
****************************************
Index(['#bócphốt', '#chiasẻ', '#gópý', '#hỏiđáp', '#lcd', '#tâmsự', '#tìmbạn',
       '#tìmngườiyêu'],
      dtype='object')
****************************************
0    4437
1     700
Name: #lcd, dtype: int64

In [96]:
result_prediction_dict = dict()
prediction_cus = model.predict(X_dev, verbose=0)
print(tokenizer.sequences_to_texts(X_dev))
# print(len(tokenizer.sequences_to_texts(X_dev)[0].split(' ')))
# print(prediction_cus)
for i in range(len(prediction_cus)):
    result_tag = Y_train.columns[np.argmax(prediction_cus[i])]
#     print(result_tag)
    result_prediction_dict[result_tag] = result_prediction_dict.get(result_tag, 0) + 1
print(result_prediction_dict)
print(max(zip(result_prediction_dict.values(), result_prediction_dict.keys()))[1])

['dạ mọi người ơi em k21 ạ', 'em muốn hỏi về chuyện học_bổng ban_đầu mới vào trường em nghe nói là điểm tb từ 7 là có tư_cách xét học_bổng', 'nhưng giờ mới biết thêm là số học_bổng là mỗi khoa xét riêng và lấy từ trên xuống', 'với sức học của em hiện_tại ráng lắm chắc cũng chỉ với đc tới 7 chấm thôi mà cái khoa em máy_tính thì toàn các vị thần không', 'các anh_chị khoá trước có_thể cho em biết là tầm bao_nhiêu điểm là mình có_thể chắc ăn có được học_bổng không ạ rồi cách ôn_thi cuối kì hiệu_quả vì điểm giữa kỳ em k được tốt lắm nên em sẽ cố_gắng hết_sức cho cuối kỳ', 'những comment góp_ý em xin chân_thành ghi_nhận ạ', 'còn về bài_ca học bách_khoa phải rớt môn học_lại thì thôi ạ vì em không muốn làm phí thêm tiền của ba_mẹ ạ', 'em cảm_ơn']
{'#hỏiđáp': 8}
#hỏiđáp


In [ ]:
print(Y_train.value_counts())

In [ ]:
print(len(Y_train))

In [ ]:
print(Y_train.loc[[228]])

In [ ]:
np.argmax(prediction[0])

In [ ]:
print(labels)

In [ ]:
Y_train.iloc[:,5]

In [ ]:
Y_train.columns

In [ ]:
Y_test.columns

In [ ]:
print(Y.value_counts())

In [ ]:
X_test[5]

In [ ]:
tokenizer.sequences_to_texts(X_test)[5]

In [39]:
print(X_dev)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 1076 1125  407  335  382  104   20 1126   56  519  102
     2   97  263    1  146  123   29   94  787  327  788  789   85 1232
   519    1  790   11  386    1   28  586  104   20  362  231 1233  519
     2  586  104   20   84   36   27   44   27    1  740    6  365   88
    18   21  177  359    5   15  407  335  162 1234  791 1235 1236  791
  1237 1023  191 1238 1239  191 1240  191 1241 1242   46  586    6    9
     8  388    2   46   36  358   81    6   48   27   25   97   21   27
  1077   83 1243 1244   51 1245  493    6   28   53  914   17   56    4
    13    2  789   14   81 1246 1247    2  324   64    6    7    1    6
    28   18  106  194   55  907  217  728    8  904   51   53    9  929
   388    2    9 1248   46    1   73   73  194    2 1249  106 12

In [45]:
print(X_test[0][np.argmax(X_test[0])])

1308


In [46]:
print(len(X_test[0]))

272


In [47]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [61]:
print(texts)

[['chào', 'mọi', 'người', 'mình', 'sinh', 'năm', '99', 'mới', 'ra', 'trường', 'mà', 'vẫn', 'chưa', 'có', 'mảnh', 'tình', 'vắt', 'vai', 'nên', 'mạo_muội', 'lên', 'đây', 'tìm', 'người_yêu'], ['sơ', 'qua', 'thì', 'mình', 'học', 'it', 'việc_làm', 'ổn_định', 'thu_nhập', '4', 'số', 'tiểu_học'], ['mình', 'khá', 'tự_ti', 'về', 'việc', 'không', 'đạt', '1', 'm7', 'và', 'hơi', 'béo', '6', 'x', 'kg', 'nên', 'mình', 'mong', 'bạn', 'nữ', 'thấp', 'hơn', 'mình', 'là', 'được'], ['các', 'bạn', 'thả', 'tim', 'mình', 'sẽ', 'inbox'], ['chào', 'mọi', 'người', 'mình', 'sinh', 'năm', '99', 'mới', 'ra', 'trường', 'mà', 'vẫn', 'chưa', 'có', 'mảnh', 'tình', 'vắt', 'vai', 'nên', 'mạo_muội', 'lên', 'đây', 'tìm', 'người_yêu'], ['sơ', 'qua', 'thì', 'mình', 'học', 'it', 'việc_làm', 'ổn_định', 'thu_nhập', '4', 'số', 'tiểu_học'], ['mình', 'khá', 'tự_ti', 'về', 'việc', 'không', 'đạt', '1', 'm7', 'và', 'hơi', 'béo', '6', 'x', 'kg', 'nên', 'mình', 'mong', 'bạn', 'nữ', 'thấp', 'hơn', 'mình', 'là', 'được'], ['các', 'bạn', '